### Handle CAMS EGG4 dataset with Zampy
Demo notebook for users and developers.

In [1]:
import numpy as np
from zampy.datasets.catalog import CAMS
from zampy.datasets.dataset_protocol import TimeBounds, SpatialBounds
from pathlib import Path

/home/yangliu/mambaforge/envs/ecoextreml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
work_dir = Path("/path_to_work_directory")
download_dir = work_dir / "download"
ingest_dir = work_dir / "ingest"
times = TimeBounds(np.datetime64("2003-01-02"), np.datetime64("2003-01-31"))
bbox_demo = SpatialBounds(54, 56, 1, 3)

Download dataset.

In [3]:
cams_dataset = CAMS()
cams_dataset.download(
    download_dir=download_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["co2_concentration"],
)

100%|██████████| 1/1 [00:00<00:00,  2.66it/s]

File 'cams_co2_concentration_2003_1-2003_1.nc' already exists, skipping...


True

Data ingestion to the unified format in `zampy`.

In [4]:
cams_dataset.ingest(download_dir, ingest_dir)

File 'cams_co2_concentration_2003_1-2003_1.nc' already exists, skipping...


True

In [5]:
ds = cams_dataset.load(
    ingest_dir=ingest_dir,
    time_bounds=times,
    spatial_bounds=bbox_demo,
    variable_names=["co2_concentration"],
    resolution=1.0,
)

In [6]:
ds

<xarray.Dataset>
Dimensions:            (time: 233, latitude: 54, longitude: 54)
Coordinates:
  * time               (time) datetime64[ns] 2003-01-02 ... 2003-01-31
  * latitude           (latitude) float64 1.0 2.0 3.0 4.0 ... 52.0 53.0 54.0
  * longitude          (longitude) float64 3.0 4.0 5.0 6.0 ... 54.0 55.0 56.0
Data variables:
    co2_concentration  (time, latitude, longitude) float32 dask.array<chunksize=(233, 54, 54), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-08-09 15:48:02 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [7]:
from zampy.datasets import converter

ds_convert = converter.convert(ds, cams_dataset, "ALMA")

co2_concentration renamed to CO2air.
Conversion of dataset 'cams' following ALMA convention is complete!


In [8]:
ds_convert

<xarray.Dataset>
Dimensions:    (time: 233, latitude: 54, longitude: 54)
Coordinates:
  * time       (time) datetime64[ns] 2003-01-02 ... 2003-01-31
  * latitude   (latitude) float64 1.0 2.0 3.0 4.0 5.0 ... 51.0 52.0 53.0 54.0
  * longitude  (longitude) float64 3.0 4.0 5.0 6.0 7.0 ... 53.0 54.0 55.0 56.0
Data variables:
    CO2air     (time, latitude, longitude) float32 dask.array<chunksize=(233, 54, 54), meta=np.ndarray>
Attributes:
    Conventions:  ALMA
    history:      2023-08-09 15:48:02 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...